# 23 Day 04

https://adventofcode.com/2023/day/4

In [3]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=4)

In [27]:
import re

class Card:
    id: int
    numbers: list[int]
    winning_numbers: list[int]

    def __init__(self, input: str) -> None:
            card_id_raw, card_raw = input.split(': ')
            self.id = int(card_id_raw.replace('Card ', ''))
            winning_raw, numbers_raw = card_raw.split(' | ')
            self.numbers = [int(n) for n in re.findall('\\d+', numbers_raw)]
            self.winning_numbers = [int(n) for n in re.findall('\\d+', winning_raw)]
    
    def __str__(self):
        # TODO need to left pad space for single digit numbers
        return f'Card {str(self.id)}: {' '.join(map(str, self.winning_numbers))} | {' '.join(map(str, self.numbers))}'

    def score(self):
        matches = set(self.winning_numbers).intersection(set(self.numbers))
        return pow(2, len(matches) - 1) if matches else 0

    @classmethod
    def fmt_cards(cls, seq) -> str:
        return '\n'.join(str(c) for c in seq)

example_input = """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"""

cards = [Card(c) for c in example_input.split('\n')]
print(Card.fmt_cards(cards))
print(f'Example Solution: {sum([c.score() for c in cards])}')
print(f'Part A Solution: {sum([Card(c).score() for c in puzzle.input_data.split('\n')])}')

Card 1: 41 48 83 86 17 | 83 86 6 31 17 9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3: 1 21 53 59 44 | 69 82 63 72 16 21 14 1
Card 4: 41 92 73 84 69 | 59 84 76 51 58 5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
Example Solution: 13
Part A Solution: 25231


In [56]:
import re
from copy import copy

class Card:
    id: int
    numbers: list[int]
    winning_numbers: list[int]

    def __init__(self, input: str) -> None:
            card_id_raw, card_raw = input.split(': ')
            self.id = int(card_id_raw.replace('Card ', ''))
            winning_raw, numbers_raw = card_raw.split(' | ')
            self.numbers = [int(n) for n in re.findall('\\d+', numbers_raw)]
            self.winning_numbers = [int(n) for n in re.findall('\\d+', winning_raw)]
    
    def __str__(self):
        # TODO need to left pad space for single digit numbers
        return f'Card {str(self.id)}: {' '.join(map(str, self.winning_numbers))} | {' '.join(map(str, self.numbers))}'

    def score(self):
        matches = set(self.winning_numbers).intersection(set(self.numbers))
        return pow(2, len(matches) - 1) if matches else 0

    @classmethod
    def fmt_cards(cls, seq) -> str:
        return '\n'.join(str(c) for c in seq)

class Game:
    cards: list[Card]

    def __init__(self, input: str) -> None:
        self.cards = [Card(c) for c in input.split('\n')]

        copies = []
        
        # I'm doing a lot of extra work because I'm assuming the card id isn't required to match the index
        for card in self.cards:
            copies.extend(self._copies(card))

        self.cards.extend(copies)
    
    def _copies(self, card: Card) -> list[Card]:
        matches = set(card.winning_numbers).intersection(set(card.numbers))
        desired_card_ids = [i for i in range(card.id + 1, card.id + len(matches) + 1)]
        # TODO should really check that I found everything I was looking for...
        copies = [copy(c) for c in self.cards if c.id in desired_card_ids]
        nested_copies = []
        for card in copies:
            nested_copies.extend(self._copies(card))
        copies.extend(nested_copies)
        return copies


    def __str__(self) -> str:
        return Card.fmt_cards(sorted(self.cards, key=lambda c: c.id))


example_input = """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"""

example_game = Game(example_input)
# print(example_game)
print(f'Example Solution B: {len(example_game.cards)}')
print(f'Part B Solution: {len(Game(puzzle.input_data).cards)}')

Example Solution B: 30
Part B Solution: 9721255


It works, but the performance is awful. A quick check with the profiler shows plenty of oppertunities for performance imporvements. I should stop deep copying the object since I'm not mutating. I also can stop pretending that the card id is not the same as the index.

    > python -m cProfile 2023_04_B.py

    204152011 function calls (194430915 primitive calls) in 253.738 seconds

    Ordered by: cumulative time

    ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        7/1    0.000    0.000  253.738  253.738 {built-in method builtins.exec}
            1    0.000    0.000  253.738  253.738 2023_04_B.py:1(<module>)
            1    0.727    0.727  253.729  253.729 2023_04_B.py:60(main)
            1    0.129    0.129  253.002  253.002 2023_04_B.py:34(__init__)
    9721255/212   33.097    0.000  252.610    1.192 2023_04_B.py:45(_copies)
    9721255  150.669    0.000  210.842    0.000 2023_04_B.py:49(<listcomp>)
    9721043   22.760    0.000   60.173    0.000 copy.py:66(copy)
    9721043   11.953    0.000   23.034    0.000 copy.py:259(_reconstruct)
    9721043    3.520    0.000    6.845    0.000 copyreg.py:104(__newobj__)

In [11]:
import re
from copy import copy

class PerformanceCard:
    id: int
    numbers: set[int]
    winning_numbers: set[int]
    won_card_ids: (int, int)

    def __init__(self, input: str) -> None:
            card_id_raw, card_raw = input.split(': ')
            self.id = int(card_id_raw.replace('Card ', ''))
            winning_raw, numbers_raw = card_raw.split(' | ')
            self.numbers = {int(n) for n in re.findall('\\d+', numbers_raw)}
            self.winning_numbers = {int(n) for n in re.findall('\\d+', winning_raw)}
            self.won_card_ids = (self.id, self.id + len(self.winning_numbers.intersection(self.numbers)))
    
    def __str__(self):
        # TODO need to left pad space for single digit numbers
        return f'Card {str(self.id)}: {' '.join(map(str, self.winning_numbers))} | {' '.join(map(str, self.numbers))}'

    @classmethod
    def fmt_cards(cls, seq) -> str:
        return '\n'.join(str(c) for c in seq)

class PerformanceGame:
    cards: list[PerformanceCard]

    def __init__(self, input: str) -> None:
        self.cards = [PerformanceCard(c) for c in input.split('\n')]

        copies = []

        for card in self.cards:
            copies.extend(self._copies(card))

        self.cards.extend(copies)
    
    def _copies(self, card: PerformanceCard) -> list[PerformanceCard]:
        copies = self.cards[card.won_card_ids[0]:card.won_card_ids[1]]
        nested_copies = []
        for card in copies:
            nested_copies = nested_copies + self._copies(card)
        return copies + nested_copies


    def __str__(self) -> str:
        return Card.fmt_cards(sorted(self.cards, key=lambda c: c.id))


example_input = """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"""

example_game = PerformanceGame(example_input)
# print(example_game)
print(f'Example Solution B: {len(example_game.cards)}')
print(f'Part B Solution: {len(PerformanceGame(puzzle.input_data).cards)}')

Example Solution B: 30
Part B Solution: 9721255


The code now runs in 3.6 seconds (longer in the profiler). I'm pretty happy with the performance gains.

    Part B Solution: 9721255
            9731067 function calls (9965 primitive calls) in 7.365 seconds

    Ordered by: cumulative time

    ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8/1    0.000    0.000    7.365    7.365 {built-in method builtins.exec}
            1    0.000    0.000    7.365    7.365 2023_04_B.py:1(<module>)
            1    0.024    0.024    7.356    7.356 2023_04_B.py:54(main)
            1    0.017    0.017    7.332    7.332 2023_04_B.py:32(__init__)
    9721255/212    7.203    0.000    7.203    0.034 2023_04_B.py:42(_copies)
        217    0.107    0.000    0.107    0.000 {method 'extend' of 'list' objects}

